In [ ]:
from torch import nn
from torch.nn.functional import relu
import torch
import os
from sklearn.model_selection import train_test_split
from random import seed, shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.nn.functional import relu
import torch
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lib.models import Gandalf
from lib.utils import *
from torch.utils.data import DataLoader
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

FOLD = 0
folds = get_leave_one_out_cv_ids_for_ekyn()
train_ids,test_ids = folds[FOLD]
print(train_ids,test_ids)
subjects = [load_eeg_label_pair(id=id,condition=condition) for id in train_ids for condition in ['Vehicle','PF']]
Xs = [subject[0] for subject in subjects]
ys = [subject[1] for subject in subjects]
train_idx,test_idx = train_test_split(range(len(subjects)*8640),test_size=.1,random_state=0,shuffle=True)
trainloader = DataLoader(dataset=SSDataset(Xs,ys,train_idx),batch_size=128,shuffle=True)
devloader = DataLoader(dataset=SSDataset(Xs,ys,test_idx),batch_size=32,shuffle=True)
del subjects
del Xs
del ys
print(f'trainloader: {len(trainloader)} batches')
print(f'devloader: {len(devloader)} batches')
model = Gandalf()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)

In [ ]:
model.to(DEVICE)
lossi = []
# for i in range(1):
for Xi,yi in tqdm(trainloader):
    Xi,yi = Xi.to(DEVICE),yi.to(DEVICE)
    logits = model(Xi)
    loss = criterion(logits,yi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lossi.append(loss.item())